In [2]:
import os
import random
import torch

from datasets import load_dataset
from transformers import AutoTokenizer

# filter the length < 7
def clean_data(example):
    tmp = re.sub(r'^(\d+).', '', example)
    tmp = re.sub(r'^\((ix|iv|v?i{0,3})\)', '', tmp)
    tmp = re.sub(r'^\((\d+)\)', '', tmp)
    new_str = tmp.translate(str.maketrans('','',string.punctuation)).replace("’s",'').replace("s’",'s')
    return new_str

# date prepared for training
def tokenize_function(examples):
    lower = [clean_data(x.lower()) for x in examples["paragraphs"]]
    lower = filter(lambda x : len(x) >7, lower )
    result = tokenizer(lower)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# split the paragraphs to sentence
# [(sent1,sent2,isnext)]
# def paragraphs_split(example):
#     nlp = stanza.Pipeline(lang='en', processors='tokenize')
#     doc = nlp(example["paragraphs"])
#     sentences = [sentence.text for sentence in doc.sentences]
#     _get_nsp_data_from_paragraph


def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # `paragraphs` is a list of lists of lists
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next


def _get_nsp_data_from_paragraph(paragraph, paragraphs, max_len=512):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # Consider 1 '<cls>' token and 2 '<sep>' tokens
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

# hklii_dataset_ch = load_dataset("json", data_files="/home/xijia/nlp/data_prepare/data/HKLII_zh.json")
# hklii_dataset_ch = hklii_dataset_ch["train"]
hklii_dataset_en = load_dataset("json", data_files="/home/huijie/legal/huggface/data_prepare/HKLII-online/HKLII.json")
hklii_dataset_en = hklii_dataset_en["train"]



Using custom data configuration default-59ce93af8ba84fa0
Reusing dataset json (/home/huijie/.cache/huggingface/datasets/json/default-59ce93af8ba84fa0/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
hklii_dataset_en[:10]

{'paragraphs': ['1.  The Plaintiff applies for leave to appeal against my decision of 8 December 2006 granting leave to the Defendant to transfer this District Court action to the Court of First Instance for the Defendant to apply for consolidation with HCA 2134 of 2005.',
  '2.  The Defendant’s application is made under S. 42 District Court Ordinance Cap. 336.  The Plaintiff’s Counsel, Mr. Ng, submitted that the Defendant’s defence has no merits and the Defendant is employing delaying tactics by applying to transfer to the Court of First Instance. ',
  '3.  Mr. Ng’s further grounds of appeal are that I have failed to take into account the Court of First Instance’s consideration of S. 43 of the District Court Ordinance, and that I have made reference to Order 15 rule 4 Rules of High Court which indicated I have taken into consideration irrelevant matters.',
  '4.  Mr. Ng further submitted that I have failed to consider the facts sufficiently and was wrong to find there were common ques

In [18]:
stanza.download('en')

NameError: name 'stanza' is not defined

hklii = train paragraphs sentences
1. clean the text 
2. 
1. shuffle paragraphs
2. get the setences in paragraph
2. for sentences in paragraphs:
    for sentence in sentences



In [3]:
nlp1 = stanza.Pipeline(lang='en', processors='tokenize')
nlp2 = stanza.Pipeline(lang='en', processors='tokenize')


def paragraphs_split(example):
    doc = nlp(example["paragraphs"])
    sentences = [sentence.text for sentence in doc.sentences]
    example["sentences"] = sentences
    return example
sampled =hklii_dataset_en.train_test_split(test_size=0.0005)
# 
# print(sampled["test"][1])
test = sampled["test"]

test = test.map(paragraphs_split,remove_columns=["ID","topic","paragraphs"],num_proc = 2)
print(test)

2022-03-29 22:31:58 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-03-29 22:31:58 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-03-29 22:32:00 INFO: Use device: gpu
2022-03-29 22:32:00 INFO: Loading: tokenize
2022-03-29 22:32:00 INFO: Use device: gpu
2022-03-29 22:32:00 INFO: Loading: tokenize
2022-03-29 22:32:13 INFO: Done loading processors!
2022-03-29 22:32:13 INFO: Done loading processors!
2022-03-29 22:32:13 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-03-29 22:32:13 INFO: Use device: gpu
2022-03-29 22:32:13 INFO: Loading: tokenize
2022-03-29 22:32:13 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-03-29 22:32:13 INFO: Use device

In [8]:
test[0]

{'sentences': ['7.  The OZP was approved on 5 December 2006.',
  'Before that date, the Land was subject to the Ngau Tam Mei Development Permission Area Plan No. DPA/YL-NTM/2 of 22 March 1994 and DPA/YL-NTM/1 of 12 July 1991 [“the DPAs”].',
  'Earlier than that, there was a Ngau Tam Mei Interim Development Permission Area Plan No. IDPA/YL-NTM/1 [“IDPA”] which was gazetted on 14 September 1990.']}

In [ ]:
def chunk_examples(examples):
    chunks = []
    for sentence in examples['sentence1']:
        chunks += [sentence[i:i + 50] for i in range(0, len(sentence), 50)]
    return {'chunks': chunks}

In [1]:
def paragraphs_split(examples):
    sentences = []
    for paragraph in examples['paragraphs']:
        tokenizer(paragraph)
        chunks += [sentence[i:i + 50] for i in range(0, len(sentence), 50)]
    return {'chunks': chunks}

["However, D2 was forced to move out from PW9's premises around the midnight of 6/7 November 2016",
 ' D2 simply had no time to give advance notice',
 ' Though the defendant had indicated his consent earlier, how D2 could ensure he would be allowed entry',
 ' The defendant liked glasses and was ordering glasses for his collection, the burglary of the optical company committed by D2 probably was not a mere coincidence, but to secure his entry into the defendant’s premises',
 '']

In [5]:
para = hklii_dataset_en["paragraphs"][2]
result = tokenizer(para)
tokenizer.decode(result.input_ids)

'[CLS] 2. the defendant [UNK] s application is made under s. 42 district court ordinance cap. 336. the plaintiff [UNK] s counsel, mr. ng, submitted that the defendant [UNK] s defence has no merits and the defendant is employing delaying tactics by applying to transfer to the court of first instance. [SEP]'

In [16]:
import spacy
from spacy.lang.en import English

nlp = spacy.load("en_core_web_sm", exclude=["parser"])
nlp.enable_pipe("senter")


In [17]:
para=  " ".join(hklii_dataset_en["paragraphs"][:10])
doc = nlp(para)
for sent in doc.sents:
    print(sent.text)

1.
 The Plaintiff applies for leave to appeal against my decision of 8 December 2006 granting leave to the Defendant to transfer this District Court action to the Court of First Instance for the Defendant to apply for consolidation with HCA 2134 of 2005.
2.  The Defendant’s application is made under S. 42 District Court Ordinance Cap.
336.
 The Plaintiff’s Counsel, Mr. Ng, submitted that the Defendant’s defence has no merits and the Defendant is employing delaying tactics by applying to transfer to the Court of First Instance.
 3.
 Mr. Ng’s further grounds of appeal are that I have failed to take into account the Court of First Instance’s consideration of S. 43 of the District Court Ordinance, and that I have made reference to Order 15 rule 4 Rules of High Court which indicated I have taken into consideration irrelevant matters.
4
.  Mr. Ng further submitted that I have failed to consider the facts sufficiently and was wrong to find there were common questions of facts and law between 

In [19]:
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize')

2022-03-30 01:22:45 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-03-30 01:22:45 INFO: Use device: gpu
2022-03-30 01:22:45 INFO: Loading: tokenize
2022-03-30 01:22:58 INFO: Done loading processors!


In [21]:
doc = nlp(para)
for sentence in doc.sentences:
    print(sentence.text)

1.  The Plaintiff applies for leave to appeal against my decision of 8 December 2006 granting leave to the Defendant to transfer this District Court action to the Court of First Instance for the Defendant to apply for consolidation with HCA 2134 of 2005.
2.  The Defendant’s application is made under S. 42 District Court Ordinance Cap. 336.
The Plaintiff’s Counsel, Mr. Ng, submitted that the Defendant’s defence has no merits and the Defendant is employing delaying tactics by applying to transfer to the Court of First Instance.
3.  Mr. Ng’s further grounds of appeal are that I have failed to take into account the Court of First Instance’s consideration of S. 43 of the District Court Ordinance, and that I have made reference to Order 15 rule 4 Rules of High Court which indicated I have taken into consideration irrelevant matters.
4.  Mr. Ng further submitted that I have failed to consider the facts sufficiently and was wrong to find there were common questions of facts and law between thi